In [1]:
import pandas as pd

In [ ]:
"""
	recebe o DataFrame e o caminho para salvar o csv
"""


def salvar_dados(df, caminho_csv):
    df.to_csv(caminho_csv, sep=";", index=False)


### Concatenando todas as baeses em uma só, já filtrando apenas os itens que serão utilizados

In [67]:
"""
	recebe o caminho de um arquivo csv
	mantém apenas as colunas e os produtos que serão utilizados
	retorna o DataFrame pronto
"""


def carregar_e_transformar_csv(csv_path):
    df = pd.read_csv(csv_path, sep=";")

    colunas_utilizadas = [
        "Estado - Sigla",
        "Municipio",
        "Produto",
        "Data da Coleta",
        "Valor de Venda",
    ]
    df = df[colunas_utilizadas]

    produtos_desejados = ["GASOLINA", "ETANOL", "DIESEL", "DIESEL S10"]
    df = df[df["Produto"].isin(produtos_desejados)]

    return df


def carregar_e_transformar_csv_encoding_latin1(csv_path):
    df = pd.read_csv(csv_path, sep=";", encoding="latin-1")

    colunas_utilizadas = [
        "Estado - Sigla",
        "Municipio",
        "Produto",
        "Data da Coleta",
        "Valor de Venda",
    ]
    df = df[colunas_utilizadas]

    produtos_desejados = ["GASOLINA", "ETANOL", "DIESEL", "DIESEL S10"]
    df = df[df["Produto"].isin(produtos_desejados)]

    return df


def carregar_e_transformar_csv_dtype_unicode(csv_path):
    df = pd.read_csv(csv_path, sep=";", dtype="unicode")

    colunas_utilizadas = [
        "Estado - Sigla",
        "Municipio",
        "Produto",
        "Data da Coleta",
        "Valor de Venda",
    ]
    df = df[colunas_utilizadas]

    produtos_desejados = ["GASOLINA", "ETANOL", "DIESEL", "DIESEL S10"]
    df = df[df["Produto"].isin(produtos_desejados)]

    return df

In [69]:
"""
	gera o DataFrame com todos os arquivos
    em seguida salva o DataFrame num novo arquivo csv
"""

df_list = []

for ano in ["13", "14", "15", "16", "17", "18", "19", "22", "23"]:
    for semestre in ["01", "02"]:
        df_list.append(carregar_e_transformar_csv(f"ca-20{ano}-{semestre}.csv"))
        
df_list.append(carregar_e_transformar_csv("ca-2020-01.csv"))
df_list.append(carregar_e_transformar_csv_dtype_unicode("ca-2020-02.csv"))
df_list.append(carregar_e_transformar_csv("ca-2021-01.csv"))
df_list.append(carregar_e_transformar_csv_encoding_latin1("ca-2021-02.csv"))       
df_list.append(carregar_e_transformar_csv("ca-2024-01.csv"))

df = pd.concat(df_list, ignore_index=True)
salvar_dados(df, "precos-combustiveis-2013-2024.csv")

### A partir da base gerada anteriormente, gera a base com as médias de valores mensais de todo o Brasil

In [18]:
"""
	recebe o DataFrame geral original
	calcula a media de cada mes, gera um novo DataFrame apenas com as colunas [ano, mes, produto, valor de venda]
    salva o DataFrame num novo arquivo csv
"""

def gerar_df_brasil(df):
    df['Data da Coleta'] = pd.to_datetime(df['Data da Coleta'], format='%d/%m/%Y')

    df['Mes'] = df['Data da Coleta'].dt.month
    df['Ano'] = df['Data da Coleta'].dt.year
    
    df['Valor de Venda'] = df['Valor de Venda'].str.replace(',', '.').astype(float)
    
    media_brasil = df.groupby(['Ano', 'Mes', 'Produto'], as_index=False)['Valor de Venda'].mean()
    
    salvar_dados(media_brasil, 'media_brasil_mes_a_mes.csv')
    
    return media_brasil

In [19]:
df = pd.read_csv("precos-combustiveis-2013-2024.csv", sep=";")
media_brasil = gerar_df_brasil(df)

### A partir da base gerada originalmente, com todos os valores, gera as bases com as médias de valores mensais para cada estado

In [ ]:
"""
	recebe o DataFrame geral original
	calcula a media de cada mes, para cada um dos estados brasileiros, gera um novo DataFrame apenas com as colunas [ano, mes, produto, valor de venda]
    salva o DataFrame num novo arquivo csv
"""


def gerar_df_estados(df):
    estados = df["Estado - Sigla"].unique()

    for estado in estados:
        df_estado = df[df["Estado - Sigla"] == estado]

        df_estado["Data da Coleta"] = pd.to_datetime(
            df_estado["Data da Coleta"], format="%d/%m/%Y"
        )

        df_estado["Mes"] = df_estado["Data da Coleta"].dt.month
        df_estado["Ano"] = df_estado["Data da Coleta"].dt.year

        df_estado["Valor de Venda"] = (
            df_estado["Valor de Venda"].str.replace(",", ".").astype(float)
        )

        media_estado = df_estado.groupby(["Ano", "Mes", "Produto"], as_index=False)[
            "Valor de Venda"
        ].mean()

        nome_arquivo = f"media_{estado}_mes_a_mes.csv"

        salvar_dados(media_estado, nome_arquivo)

In [ ]:
gerar_df_estados(pd.read_csv("precos-combustiveis-2013-2024.csv", sep=";"))

In [ ]:
def gerar_media_mensal_estado_municipio(estado, municipio, arquivo_base):
    df = pd.read_csv(arquivo_base, sep=";")
    df_filtrado = df[(df["Estado - Sigla"] == estado) & (df["Municipio"] == municipio)]
    df_filtrado["Data da Coleta"] = pd.to_datetime(df_filtrado["Data da Coleta"], format='%d/%m/%Y')
    df_filtrado["Ano"] = df_filtrado["Data da Coleta"].dt.year
    df_filtrado["Mes"] = df_filtrado["Data da Coleta"].dt.month
    df_filtrado["Valor de Venda"] = df_filtrado["Valor de Venda"].str.replace(',', '.').astype(float)
    df_media_mensal = df_filtrado.groupby(["Ano", "Mes", "Produto"], as_index=False)["Valor de Venda"].mean()
    nome_arquivo_saida = f"media_{estado}_{municipio}_mes_a_mes.csv"
    df_media_mensal.to_csv(nome_arquivo_saida, sep=";", index=False)
    print(f"Arquivo {nome_arquivo_saida} gerado com sucesso!")

In [ ]:
gerar_media_mensal_estado_municipio("PR", "CASCAVEL", "precos-combustiveis-2013-2024.csv")

Arquivo media_PR_CASCAVEL_mes_a_mes.csv gerado com sucesso!


C:\Users\Pedro\AppData\Local\Temp\ipykernel_8620\769203073.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["Data da Coleta"] = pd.to_datetime(df_filtrado["Data da Coleta"], format='%d/%m/%Y')
C:\Users\Pedro\AppData\Local\Temp\ipykernel_8620\769203073.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["Ano"] = df_filtrado["Data da Coleta"].dt.year
C:\Users\Pedro\AppData\Local\Temp\ipykernel_8620\769203073.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [ ]:
df_all = pd.read_csv("precos-combustiveis-2013-2024.csv", sep=";")

In [ ]:
df_all.info()
print(len(df_all))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10393358 entries, 0 to 10393357
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   Estado - Sigla  object
 1   Municipio       object
 2   Produto         object
 3   Data da Coleta  object
 4   Valor de Venda  object
dtypes: object(5)
memory usage: 396.5+ MB
10393358


In [23]:
"""
	recebe o DataFrame geral original
	calcula a media de cada mes, para cada um dos estados brasileiros, gera um novo DataFrame apenas com as colunas [ano, mes, produto, valor de venda]
    salva o DataFrame num novo arquivo csv
"""


def gerar_df_estados(df):
    estados = df["Estado - Sigla"].unique()

    for estado in estados:
        df_estado = df[df["Estado - Sigla"] == estado]

        df_estado["Data da Coleta"] = pd.to_datetime(
            df_estado["Data da Coleta"], format="%d/%m/%Y"
        )

        df_estado["Mes"] = df_estado["Data da Coleta"].dt.month
        df_estado["Ano"] = df_estado["Data da Coleta"].dt.year

        df_estado["Valor de Venda"] = (
            df_estado["Valor de Venda"].str.replace(",", ".").astype(float)
        )

        media_estado = df_estado.groupby(["Ano", "Mes", "Produto"], as_index=False)[
            "Valor de Venda"
        ].mean()

        nome_arquivo = f"media_{estado}_mes_a_mes.csv"

        salvar_dados(media_estado, nome_arquivo)

In [ ]:
gerar_df_estados(pd.read_csv("precos-combustiveis-2013-2024.csv", sep=";"))

In [14]:
def gerar_media_mensal_estado_municipio(estado, municipio, arquivo_base):
    df = pd.read_csv(arquivo_base, sep=";")
    df_filtrado = df[(df["Estado - Sigla"] == estado) & (df["Municipio"] == municipio)]
    df_filtrado["Data da Coleta"] = pd.to_datetime(df_filtrado["Data da Coleta"], format='%d/%m/%Y')
    df_filtrado["Ano"] = df_filtrado["Data da Coleta"].dt.year
    df_filtrado["Mes"] = df_filtrado["Data da Coleta"].dt.month
    df_filtrado["Valor de Venda"] = df_filtrado["Valor de Venda"].str.replace(',', '.').astype(float)
    df_media_mensal = df_filtrado.groupby(["Ano", "Mes", "Produto"], as_index=False)["Valor de Venda"].mean()
    nome_arquivo_saida = f"media_{estado}_{municipio}_mes_a_mes.csv"
    df_media_mensal.to_csv(nome_arquivo_saida, sep=";", index=False)
    print(f"Arquivo {nome_arquivo_saida} gerado com sucesso!")

In [15]:
gerar_media_mensal_estado_municipio("PR", "CASCAVEL", "precos-combustiveis-2013-2024.csv")

Arquivo media_PR_CASCAVEL_mes_a_mes.csv gerado com sucesso!


C:\Users\Pedro\AppData\Local\Temp\ipykernel_8620\769203073.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["Data da Coleta"] = pd.to_datetime(df_filtrado["Data da Coleta"], format='%d/%m/%Y')
C:\Users\Pedro\AppData\Local\Temp\ipykernel_8620\769203073.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado["Ano"] = df_filtrado["Data da Coleta"].dt.year
C:\Users\Pedro\AppData\Local\Temp\ipykernel_8620\769203073.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [3]:
df_all = pd.read_csv("precos-combustiveis-2013-2024.csv", sep=";")

In [5]:
df_all.info()
print(len(df_all))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10393358 entries, 0 to 10393357
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   Estado - Sigla  object
 1   Municipio       object
 2   Produto         object
 3   Data da Coleta  object
 4   Valor de Venda  object
dtypes: object(5)
memory usage: 396.5+ MB
10393358


### A partir da base gerada originalmente, com todos os valores, gera as bases com as médias de valores mensais para cada estado